## VggNet

In [2]:
# 导入相关的包
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import os

In [3]:
# 对图片的预处理
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor()
])

# 导入数据集
train_data = torchvision.datasets.CIFAR10(root='../数据集/cifar10', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='../数据集/cifar10', train=False, transform=transform, download=True)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
# 保存数据集的长度
train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

训练数据集的长度为：50000
测试数据集的长度为：10000


In [5]:
# 创建数据加载器
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [7]:
# 定义vgg块
# num_convs：卷积层的个数
# in_channels：输入通道数
# out_channels：输出通道数
def vgg_block(num_convs, in_channels, out_channels):
    layers = [] # 创建一个列表，用于存放vgg块中的卷积层和池化层
    for i in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [12]:
# vgg块的参数
conv_arch = ((1,64), (1,128), (2,256), (2,512), (2,512))
# vgg模型
def vgg(conv_arch):
    conv_blocks = [] # 创建一个列表，用于存放vgg块
    in_channels = 3 # 输入通道数
    # 遍历conv_arch，创建vgg块
    for (num_convs, out_channels) in conv_arch:
        conv_blocks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels
    
    return nn.Sequential(
        *conv_blocks,
        nn.Flatten(),
        nn.Linear(out_channels*7*7, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 10)
    )

vgg11 = vgg(conv_arch)

In [14]:
# 观察每一层输出的形状
x = torch.randn(size=(1, 3, 224, 224))
for blk in vgg11:
    x = blk(x)
    print(blk.__class__.__name__, 'output shape: \t\t', x.shape)

Sequential output shape: 		 torch.Size([1, 64, 112, 112])
Sequential output shape: 		 torch.Size([1, 128, 56, 56])
Sequential output shape: 		 torch.Size([1, 256, 28, 28])
Sequential output shape: 		 torch.Size([1, 512, 14, 14])
Sequential output shape: 		 torch.Size([1, 512, 7, 7])
Flatten output shape: 		 torch.Size([1, 25088])
Linear output shape: 		 torch.Size([1, 4096])
ReLU output shape: 		 torch.Size([1, 4096])
Dropout output shape: 		 torch.Size([1, 4096])
Linear output shape: 		 torch.Size([1, 4096])
ReLU output shape: 		 torch.Size([1, 4096])
Dropout output shape: 		 torch.Size([1, 4096])
Linear output shape: 		 torch.Size([1, 10])
